Enregistre le csv dans un S3 Bucket et crée la base de données Postgresql

In [ ]:
import boto3
import botocore
import os
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, DateTime, MetaData, ForeignKey, delete
from variables import cities_csv_name, hotels_jl_name, weather_csv_name, bucket_csv_name, final_csv_name

## Connection AWS
print("Connection AWS ...")
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"], aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.resource("s3")
s3_client = session.client("s3")
print("Connecté !")


## Création bucket S3
print("Création bucket ...")

bucket_name = "dsfs34-kayak-mj"

# Vérifier si le bucket existe
def bucket_exists(name):
    try:
        s3_client.head_bucket(Bucket=name)
        return True
    except botocore.exceptions.ClientError as e:
        error_code = e.response["Error"]["Code"]
        if error_code in ("404", "NoSuchBucket"):
            return False
        raise

if not bucket_exists(bucket_name):
    bucket = s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={"LocationConstraint": "eu-west-3"})
else:
    bucket = s3.Bucket(bucket_name)
    
# Upload fichier CSV
bucket.upload_file(final_csv_name, bucket_csv_name)
print("Bucket créé !")


## Moteur qui assure la connection BDD / Python
print("Connection BDD RDS ...")
engine = create_engine(f"postgresql+psycopg2://{os.environ["DB_USERNAME"]}:{os.environ["DB_PASSWORD"]}@{os.environ["DB_HOSTNAME"]}/{os.environ["DB_NAME"]}", echo=True)
meta = MetaData()
conn = engine.connect()
print("Connecté !")

# Table "cities"
table_cities = Table(
    "cities", meta,
    Column("city_id", Integer, primary_key = True),
    Column("name", String),
    Column("latitude", String),
    Column("longitude", String)
)

# Table "weathers"
table_weathers = Table(
    "weathers", meta,
    Column("weather_id", Integer, primary_key = True),
    Column("city_id", Integer, ForeignKey("cities.city_id")),
    Column("date", DateTime),
    Column("temperature", Float),
    Column("temperature_feels_like", Float),
    Column("condition", String),
    Column("wind_speed", Float),
    Column("proba_precipitation", Float)
)

# Table "hotels"
table_hotels = Table(
    "hotels", meta,
    Column("hotel_id", Integer, primary_key = True),
    Column("city_id", Integer, ForeignKey("cities.city_id")),
    Column("name", String),
    Column("link", String),
    Column("score", Float),
    Column("description", String),
    Column("latitude", String),
    Column("longitude", String)
)

# Création des tables
meta.create_all(engine)

# On vide les tables avant de les remplir
stmt_delete_weathers = delete(table_weathers)
stmt_delete_hotels = delete(table_hotels)
stmt_delete_cities = delete(table_cities)
conn.execute(stmt_delete_weathers)
conn.execute(stmt_delete_hotels)
conn.execute(stmt_delete_cities)
conn.commit()

# Villes = 1er dataframe --> Table cities
df_cities = pd.read_csv(cities_csv_name)
# On renomme les colonnes du dataframe pour qu'elles correspondent aux colonnes de la table
df_cities = df_cities.rename({"id":"city_id", "city":"name", "lat":"latitude", "lon":"longitude"}, axis = 1)
df_cities = df_cities.set_index("city_id")
df_cities.to_sql("cities", engine, if_exists = "append")

# Météo = 2ème dataframe --> Table weathers
df_weather = pd.read_csv(weather_csv_name)
df_weather.index.names = ["weather_id"]
# id BDD commence à 1
df_weather.index += 1
# On renomme les colonnes du dataframe pour qu'elles correspondent aux colonnes de la table
df_weather = df_weather.rename({"id_city":"city_id"}, axis = 1)
df_weather.to_sql("weathers", engine, if_exists = "append")

# Hotels = 3ème DataFrame --> Table hotels
df_hotels = pd.read_json(hotels_jl_name, lines = True)
df_hotels.index.names = ["hotel_id"]
# id BDD commence à 1
df_hotels.index += 1
# On renomme les colonnes du dataframe pour qu'elles correspondent aux colonnes de la table
df_hotels = df_hotels.drop(columns=["city"])
df_hotels["hotel_score"] = df_hotels["hotel_score"].str.replace(",", ".")
df_hotels = df_hotels.rename({"id_city":"city_id", "hotel_name":"name", "hotel_link":"link", 
                                "hotel_score":"score", "hotel_description":"description", "hotel_lat":"latitude", "hotel_lon":"longitude"}, axis = 1)
df_hotels.to_sql("hotels", engine, if_exists = "append")